In [1]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
import warnings

warnings.filterwarnings(action='ignore')

url = 'https://gol.gg/players/list/season-ALL/split-ALL/tournament-LCK%20Spring%202022/'

driver = webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver.get(url)

df = pd.DataFrame()

columns_list = [
    '선수명',
    '포지션',
    '게임수',
    '승률',
    'KDA',
    '경기당 평균 킬',
    '경기당 평균 데스',
    '경기당 평균 어시',
    '분당 CS',
    '분당 골드',
    '킬 관여율',
    '경기당 데미지 비중',
    '분당 데미지',
    '분당 시야점수',
    '분당 와드',
    '분당 와드제거',
    '분당 와드에 노출',
    '15분 골드격차',
    '15분 CS 격차',
    '15분 경험치 격차',
    '퍼블 관여율',
    '퍼블 희생',
    '펜타킬',
    '솔로킬'
]

table = driver.find_element(By.CLASS_NAME, 'table_list').text.split('\n')
data = [i for i in table]
cnt = len([i for i in data[1].split(' ') if i not in ['']])
data = pd.DataFrame(data[1:], columns=['test'])
load_data = data.test.str.split(' ').str
dc = 0

for i in range(cnt):
    if load_data[i][0] != '':
        df[columns_list[dc]] = load_data[i]
        dc = dc + 1
    
while 1:
    try:
        driver.find_element(By.XPATH, '/html/body/div/main/div[2]/div/div[2]/div/div[2]/div[1]/a[2]').click()
        table = driver.find_element(By.CLASS_NAME, 'table_list').text.split('\n')
        data = [i for i in table]
        data = pd.DataFrame(data[1:], columns=['test'])
        load_data = data.test.str.split(' ').str[-1]
        df[columns_list[cnt]] = load_data
        cnt = cnt + 1
    except:
        break
        
for i in columns_list:
    try:
        df.loc[df[i] == '-', i] = 0
    except:
        pass
        
for i in columns_list:
    try:
        df.loc[df[i] == '', i] = 0
    except:
        pass        

edit_columns = ['승률','킬 관여율','퍼블 관여율','퍼블 희생']

for i in range(len(edit_columns)):
    df[edit_columns[i]] = [round(float(i[:-1])/100,3) for i in df[edit_columns[i]]]
    
top_data = df[df['포지션'] == 'TOP']
jg_data = df[df['포지션'] == 'JUNGLE']
mid_data = df[df['포지션'] == 'MID']
adc_data = df[df['포지션'] == 'ADC']
sup_data = df[df['포지션'] == 'SUPPORT']

gb_params = {
    'learning_rate' : [0.1,0.2,0.3], # 기본 0.1
    'n_estimators' : [50,100,150], # 기본 100
    'min_samples_split' : [1,2,3], # 기본 2
    'min_samples_leaf' : [1,2,3], # 기본 1
    'max_depth' : [5,6,7] # 기본 6
}

driver.quit()

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

def model_train(data):
    while 1:
        X = data.drop(['선수명','포지션','승률','게임수'],axis=1).values.astype('float')
        X = StandardScaler().fit_transform(X)
        y = (data['승률'].values > 0.5).astype('int')

        gb = GradientBoostingClassifier(random_state=0)

        grid_cv = GridSearchCV(gb, param_grid=gb_params, cv=3, n_jobs= -1)
        grid_cv.fit(X,y)
        bpv = list(grid_cv.best_params_.values())
        model = GradientBoostingClassifier(learning_rate=bpv[0], max_depth=bpv[1], min_samples_leaf=bpv[2], min_samples_split=bpv[3], n_estimators=bpv[4])
        model.fit(X,y)
        if model.score(X,y) == 1.0:
            return model

top = model_train(top_data)
jg = model_train(jg_data)
mid = model_train(mid_data)
adc = model_train(adc_data)
sup = model_train(sup_data)

def make_score(model, result):
    result['점수'] = 0
    for i,j in zip(result.drop(['선수명','포지션','승률','게임수'],axis=1).columns, list(model.feature_importances_)):
        result['점수'] = result['점수'] + result[i].astype('float') * j
    result['점수'] = StandardScaler().fit_transform(result[['점수']])
    result['점수'] = round(result['점수'],2)
    return result

df_top = make_score(top, top_data)
df_jg = make_score(jg, jg_data)
df_mid = make_score(mid, mid_data)
df_adc = make_score(adc, adc_data)
df_sup = make_score(sup, sup_data)

from tabulate import tabulate
    
a = tabulate(
    tabular_data=df_top[['선수명','포지션','점수']].sort_values(by=['점수'], axis=0, ascending=0),
    headers=['선수명','포지션','점수'],
    tablefmt='psql'
)
b = tabulate(
    tabular_data=df_jg[['선수명','포지션','점수']].sort_values(by=['점수'], axis=0, ascending=0),
    headers=['선수명','포지션','점수'],
    tablefmt='psql'
)
c = tabulate(
    tabular_data=df_mid[['선수명','포지션','점수']].sort_values(by=['점수'], axis=0, ascending=0),
    headers=['선수명','포지션','점수'],
    tablefmt='psql'
)
d = tabulate(
    tabular_data=df_adc[['선수명','포지션','점수']].sort_values(by=['점수'], axis=0, ascending=0),
    headers=['선수명','포지션','점수'],
    tablefmt='psql'
)
e = tabulate(
    tabular_data=df_sup[['선수명','포지션','점수']].sort_values(by=['점수'], axis=0, ascending=0),
    headers=['선수명','포지션','점수'],
    tablefmt='psql'
)

print(a)
print(b)
print(c)
print(d)
print(e)

DWG_KIA = ['Burdol','Hoya','Canyon','ShowMaker','deokdam','Kellin']
T1 = ['Zeus','Oner','Faker','Gumayusi','Keria','Asper']
Gen_G = ['Doran','Peanut','YoungJae','Chovy','Ruler','Lehends']
NongShim_REDFORCE = ['Canna','Dread','Bdd','Ghost','Peter','Effort']
Liiv_SANDBOX = ['Dove','Howling','Croco','Clozer','Envyy','Ice','Kael']
Kwangdong_Freecs = ['Kiin','Ellim','FATE','Teddy','Leo','Hoit','MapSSi']
KT_Rolster = ['Rascal','Cuzz','GIDEON','Aria','Aiming','Life']
Hanwha_Life_Esports = ['DuDu','OnFleek','Willer','Karis','SamD','Vsta']
Fredit_BRION = ['Morgan','Sw0rd','UmTi','Lava','Hena','Delight']
DRX = ['kingen','Pyosik','Zeka','Deft','Taeyoon','BeryL']

alls = pd.concat([df_top,df_jg,df_mid,df_adc,df_sup])

def rosters(player_list, team):
    p_df = pd.DataFrame()
    for i in player_list:
        p_df = p_df.append(alls[alls['선수명'] == i])
        
    p_df = p_df.append({'선수명':team,'포지션':'총점','점수':round(sum(p_df['점수']),2)}, ignore_index=True)
        
    a = tabulate(
        tabular_data=p_df[['선수명','포지션','점수']],
        headers=['선수명','포지션','점수'],
        tablefmt='psql'
    )
    print(a)
    
rosters(DWG_KIA,'DWG_KIA')
rosters(T1,'T1')
rosters(Gen_G,'Gen_G')
rosters(NongShim_REDFORCE,'NongShim_REDFORCE')
rosters(Liiv_SANDBOX,'Liiv_SANDBOX')
rosters(Kwangdong_Freecs,'Kwangdong_Freecs')
rosters(KT_Rolster,'KT_Rolster')
rosters(Hanwha_Life_Esports,'Hanwha_Life_Esports')
rosters(Fredit_BRION,'Fredit_BRION')
rosters(DRX,'DRX')

+----+----------+----------+--------+
|    | 선수명   | 포지션   |   점수 |
|----+----------+----------+--------|
| 13 | Doran    | TOP      |   1.49 |
|  5 | Canna    | TOP      |   1.37 |
| 32 | Kiin     | TOP      |   1.18 |
| 42 | Rascal   | TOP      |   0.46 |
| 50 | Zeus     | TOP      |   0.4  |
| 36 | Morgan   | TOP      |  -0.23 |
| 16 | DuDu     | TOP      |  -0.32 |
| 14 | Dove     | TOP      |  -0.83 |
|  4 | Burdol   | TOP      |  -0.89 |
| 26 | Hoya     | TOP      |  -1.15 |
| 52 | kingen   | TOP      |  -1.49 |
+----+----------+----------+--------+
+----+----------+----------+--------+
|    | 선수명   | 포지션   |   점수 |
|----+----------+----------+--------|
| 38 | Oner     | JUNGLE   |   1.36 |
| 39 | Peanut   | JUNGLE   |   1.36 |
| 10 | Cuzz     | JUNGLE   |   0.95 |
|  6 | Canyon   | JUNGLE   |   0.83 |
| 15 | Dread    | JUNGLE   |  -0.11 |
| 18 | Ellim    | JUNGLE   |  -0.4  |
| 37 | OnFleek  | JUNGLE   |  -0.69 |
| 47 | UmTi     | JUNGLE   |  -0.75 |
|  9 | Croco    | JUNGLE   |